# Setting Up Chameleon Environment for Serving
- site being used here is
- Model Serving in

In [1]:
from chi import server, context, lease
import os

context.version = "1.0"
context.choose_project()
context.choose_site(default="CHI@UC")

In [2]:
l = lease.get_lease(f"project_33")
l.show()

HTML(value='\n        <h2>Lease Details</h2>\n        <table>\n            <tr><th>Name</th><td>project_33</td…

Lease Details:
Name: project_33
ID: dd3c9ae5-5b26-4b54-a786-9d40ae49d1bb
Status: ACTIVE
Start Date: 2025-05-10 07:15:00
End Date: 2025-05-10 13:50:00
User ID: 1f8750751b44afe6b2d31ff1c8647770e649270f1930abf543976a163d96d5b2
Project ID: 7c0a7a1952e44c94aa75cae1ff5dc9b4

Node Reservations:
ID: 573e46c9-0bb4-4efd-a3d0-d203514afd60, Status: active, Min: 1, Max: 1

Floating IP Reservations:

Network Reservations:

Events:


In [3]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-serve-model-{username}",
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

Waiting for server node-serve-model-ns6108_nyu_edu's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-serve-model-ns6108_nyu_edu
Id,65914bd7-cd2a-4968-91d9-2427f0d64313
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.140.82.70 (v4) Type: fixed MAC: 14:23:f2:a3:ef:f0
Network Name,sharednet1
Created At,2025-05-10T07:15:20Z
Keypair,trovi-ae48ff0
Reservation Id,573e46c9-0bb4-4efd-a3d0-d203514afd60
Host Id,c15c5d0cd98629a41c320d11364f137f4320899eed52f609fb88500c


In [4]:
s.associate_floating_ip()

In [5]:
s.refresh()
s.check_connectivity()

Checking connectivity to 192.5.87.181 port 22.


Connection successful


In [6]:
s.refresh()
s.show(type="widget")

Attribute,node-serve-model-ns6108_nyu_edu
Id,65914bd7-cd2a-4968-91d9-2427f0d64313
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.140.82.70 (v4) Type: fixed MAC: 14:23:f2:a3:ef:f0 IP: 192.5.87.181 (v4) Type: floating MAC: 14:23:f2:a3:ef:f0
Network Name,sharednet1
Created At,2025-05-10T07:15:20Z
Keypair,trovi-ae48ff0
Reservation Id,573e46c9-0bb4-4efd-a3d0-d203514afd60
Host Id,c15c5d0cd98629a41c320d11364f137f4320899eed52f609fb88500c


# Git Clone LLM Summarization

In [7]:
s.execute("git clone https://github.com/shettynitis/LLM_LegalDocSummarization.git")
s.execute("cd LLM_LegalDocSummarization && git checkout Serving")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 192.5.87.181: b'fe071e36f39c4af442594432be41105c'
  warnings.warn(
Cloning into 'LLM_LegalDocSummarization'...
Switched to a new branch 'Serving'


branch 'Serving' set up to track 'origin/Serving'.


<Result cmd='cd LLM_LegalDocSummarization && git checkout Serving' exited=0>

# Setting Up Docker

In [8]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c doc

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

# Set up the NVIDIA container toolkit

In [9]:
s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor --batch --yes -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg && \
curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list > /dev/null")
s.execute("sudo apt update")
s.execute("sudo apt-get install -y nvidia-container-toolkit")
s.execute("sudo nvidia-ctk runtime configure --runtime=docker")
# for https://github.com/NVIDIA/nvidia-container-toolkit/issues/48
s.execute("sudo jq 'if has(\"exec-opts\") then . else . + {\"exec-opts\": [\"native.cgroupdriver=cgroupfs\"]} end' /etc/docker/daemon.json | sudo tee /etc/docker/daemon.json.tmp > /dev/null && sudo mv /etc/docker/daemon.json.tmp /etc/docker/daemon.json")
s.execute("sudo systemctl restart docker")

Hit:1 https://download.docker.com/linux/ubuntu noble InRelease
Get:2 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease [1477 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Get:4 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  Packages [18.6 kB]
Hit:5 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Fetched 528 kB in 1s (529 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
4 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libnvidia-container-tools libnvidia-co

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 5849 kB in 1s (5820 kB/s)
Selecting previously unselected package libnvidia-container1:amd64.
(Reading database ... 113813 files and directories currently installed.)
Preparing to unpack .../libnvidia-container1_1.17.6-1_amd64.deb ...
Unpacking libnvidia-container1:amd64 (1.17.6-1) ...
Selecting previously unselected package libnvidia-container-tools.
Preparing to unpack .../libnvidia-container-tools_1.17.6-1_amd64.deb ...
Unpacking libnvidia-container-tools (1.17.6-1) ...
Selecting previously unselected package nvidia-container-toolkit-base.
Preparing to unpack .../nvidia-container-toolkit-base_1.17.6-1_amd64.deb ...
Unpacking nvidia-container-toolkit-base (1.17.6-1) ...
Selecting previously unselected package nvidia-container-toolkit.
Preparing to unpack .../nvidia-container-toolkit_1.17.6-1_amd64.deb ...
Unpacking nvidia-container-toolkit (1.17.6-1) ...
Setting up nvidia-container-toolkit-base (1.17.6-1) ...
Setting up libnvidia-container1:amd64 (1.17.6-1) ...
Setting up lib

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
time="2025-05-10T07:28:06Z" level=info msg="Config file does not exist; using empty config"
time="2025-05-10T07:28:06Z" level=info msg="Wrote updated config to /etc/docker/daemon.json"
time="2025-05-10T07:28:06Z" level=info msg="It is recommended that docker daemon be restarted."


<Result cmd='sudo systemctl restart docker' exited=0>

In [10]:
s.execute("sudo apt update")
s.execute("sudo apt -y install nvtop")

Hit:1 https://download.docker.com/linux/ubuntu noble InRelease
Hit:2 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Fetched 508 kB in 1s (500 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
4 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  nvtop
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 62.8 kB of archives.
After this operation, 180 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble/multiverse amd64 nvtop amd64 3.0.2-1 [62.8 kB]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 62.8 kB in 0s (151 kB/s)
Selecting previously unselected package nvtop.
(Reading database ... 113837 files and directories currently installed.)
Preparing to unpack .../nvtop_3.0.2-1_amd64.deb ...
Unpacking nvtop (3.0.2-1) ...
Setting up nvtop (3.0.2-1) ...
Processing triggers for man-db (2.12.0-4build2) ...


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.


<Result cmd='sudo apt -y install nvtop' exited=0>

In [11]:
s.execute("nvidia-smi")


Sat May 10 07:28:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000000:25:00.0 Off |                    0 |
| N/A   53C    P0             49W /  300W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

<Result cmd='nvidia-smi' exited=0>

# Open an SSH session

Finally, open an SSH sesson on your server. From your local terminal, run

``ssh -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D``

where,

in place of ~/.ssh/id_rsa_chameleon, substitute the path to our project key that you had uploaded

in place of A.B.C.D, use the floating IP address you just associated to your instance.

# Prepare data

- On the compute instance, install rclone:
````
# run on node-persist
curl https://rclone.org/install.sh | sudo bash
````
````
# run on node-persist
# this line makes sure user_allow_other is un-commented in /etc/fuse.conf
sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf
````
````
# run on node-persist
mkdir -p ~/.config/rclone
nano  ~/.config/rclone/rclone.conf # Add the file Sakshi sent
rclone lsd chi_tacc:

````
- Now present node Object Volume
````
# run on node-persist
sudo mkdir -p /mnt/LlamaData
sudo chown -R cc /mnt/LlamaData
sudo chgrp -R cc /mnt/LlamaData

# run on node-persist
rclone mount chi_tacc:object-persist-project33 /mnt/LlamaData --read-only --allow-other --daemon
  
````

# Launch a Jupyter container

````
# run on node server
docker build -t jupyter-onnx -f LLM_LegalDocSummarization/Serving/Docker/Dockerfile.jupyter-onnx-gpu .

````


````
# run on node server
docker run  -d --rm  -p 8888:8888 \
    --gpus all \
    --shm-size 16G \
    -e LLAMA_DATA_DIR=/mnt/LlamaData/ \
    -v ~/LLM_LegalDocSummarization:/home/jovyan/work \
    --mount type=bind,source=/mnt/LlamaData/,target=/mnt/LlamaData/,readonly \
    --name jupyter \
    jupyter-onnx

````

````
# run on node server
docker logs jupyter
````

and look for a line like
````
http://127.0.0.1:8888/lab?token=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
````